In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Prepare basic parameters

In [5]:
# define the years to be classified
year_name  = list([f'{span[0]}_{span[1]}' for span in zip(range(1990,2020,3),range(1992,2020,3)) ])[::-1]

# import north_china_plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [8]:
year_name

['2017_2019',
 '2014_2016',
 '2011_2013',
 '2008_2010',
 '2005_2007',
 '2002_2004',
 '1999_2001',
 '1996_1998',
 '1993_1995',
 '1990_1992']

##### Prepare input_image for classification

In [9]:
# import Landsat img
Landsat_img = [ee.Image(f"users/wang8052664/Cloud_Free_Img/Landsat_cloud_free_{year}")\
                 .clip(North_China_Plain)  for year in year_name]

# import Fourier img   
Fourier_img = [ee.Image(f"users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_{year}_harmonic_3")\
                 .clip(North_China_Plain)  for year in year_name[5:]] + \
              [ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year}_harmonic_3")\
                 .clip(North_China_Plain)  for year in year_name[:5]] 

# Import the Index-mean Img
NDVI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDVI").rename('Mean_NDVI')  
            for year in year_name ]
NDBI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDBI").rename('Mean_NDBI')  
            for year in year_name ]
EVI_img  = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_EVI").rename('Mean_EVI')  
            for year in year_name ]

# stack Landsat_Fourier_Nomalized together
Stack_img = [ee.Image(img) for img in zip(Landsat_img,Fourier_img,NDVI_img,NDBI_img,EVI_img)]

##### Prepare input sample points

In [31]:
path = 'users/Jinzhu_Deakin/North_China_Plain/Sample_with_Landsat_Fourier_Normalized'

# Option_1: Use the Invarient points
Invarient_non_built_sample = [ee.FeatureCollection(f'{path}/Invarient_non_built_{year}')\
                                .filterMetadata('Built','equals',0) for year in year_name]
Invarient_built_sample     = [ee.FeatureCollection(f'{path}/Invarient_built_{year}')
                                for year in year_name]
Invarient_sample_merge     = [i[0].merge(i[1]) for i in zip(Invarient_non_built_sample,Invarient_built_sample)]



# Option_2: Use the Each_year points
Each_year_non_built_sample = [ee.FeatureCollection(f'{path}/Each_year_visual_check_{year}')
                                for year in year_name[:5]]
Each_year_built_sample     = [ee.FeatureCollection(f'{path}/Each_year_built_{year}')
                                for year in year_name[:5]]
Each_year_sample_merge     = [i[0].merge(i[1]) for i in zip(Each_year_non_built_sample,Each_year_built_sample)]




# Option_3: Use the Invarient-non-buil and Invarient-Zoning sample together as built point
Zone_invarient_built = [i[0].merge(i[1]) for i in zip([ee.FeatureCollection(f'{path}/Each_year_built_{year}')
                                                                       for year in year_name],
                                                      Invarient_built_sample)]
# limit the Zone_invarient_built point to 8000
Zone_invarient_built_limit = [i.randomColumn('x').limit(8000,'x') for i in Zone_invarient_built]
Zone_Invarient_merge = [i[0].merge(i[1]) for i in zip(Invarient_non_built_sample,Zone_invarient_built_limit)]

##### Stack input_img and input_sample together

In [33]:
# first put all inputs together
stack_name   = ['Invarient'] * len(Invarient_sample_merge) + \
               ['Each_year'] * len(Each_year_sample_merge) + \
               ['Zone_invarient'] * len(Zone_Invarient_merge)
stack_year   = year_name + year_name[:5] + year_name
stack_img    = Stack_img + Stack_img[:5] + Stack_img
stack_sample = Invarient_sample_merge + Each_year_sample_merge + Zone_Invarient_merge

# stack all ingredients together
Stack_img_sample = [(stack_name[i],stack_year[i],stack_img[i],stack_sample[i]) for i in range(len(stack_name))]

In [36]:
Classificatioin_result = []

for input_variable in Stack_img_sample:
    
    classificaiton_tpye = input_variable[0]
    year = input_variable[1]
    sample_pt = input_variable[3]
    input_img = input_variable[2]
    input_band = input_img.bandNames().getInfo()

    # Instatiate the class with a name.
    classification = Classification(year_name      = year,
                                    Verified_point = sample_pt,
                                    Input_img      = input_img,
                                    Input_band     = input_band,
                                    Tree_num       = 100,
                                    Zone_sample    = None,
                                    classProperty  = 'Built')


    # perform the classification on Input_img and Samples
    classification.Stp_1_Classification_on_img()
    classification.Stp_2_Classification_on_Samples()

    # Get the classified img
    classified_img = classification.classification_img

    # Get the classified samples
    train_sample_classified        = classification.Train_sample_classification
    test_sample_classified         = classification.Test_sample_classification
    
    # add classfication into the list
    Classificatioin_result.append((classificaiton_tpye,year,classified_img,test_sample_classified))

##### Export the result

In [24]:
Classificatioin_result[4:7]

[('Invarient',
  '2005_2007',
  <ee.featurecollection.FeatureCollection at 0x7fed1bbae860>),
 ('Invarient',
  '2002_2004',
  <ee.featurecollection.FeatureCollection at 0x7fed1ba420f0>),
 ('Invarient',
  '1999_2001',
  <ee.featurecollection.FeatureCollection at 0x7fed1ba4e630>)]

In [12]:
for item in Classificatioin_result[4:7]:
    
    asset_path = 'users/Jinzhu_Deakin/North_China_Plain/Classified_img'
    
    Classification_type = item[0]
    year = item[1]
    img = item[2]
    
    export_name = f'{Classification_type}_{year}'
    
    task = ee.batch.Export.image.toAsset(   image        = img,
                                            description  = export_name,
                                            assetId      = f'{asset_path}/{export_name}',
                                            region       = North_China_Plain.geometry().bounds(),
                                            scale        = 30,
                                            maxPixels    = int(1e13))
    task.start()
    
    print(f'Exporting {export_name} successful!')

Exporting Invarient_2005_2007 successful!
Exporting Invarient_2002_2004 successful!
Exporting Invarient_1999_2001 successful!


In [37]:
df = pd.DataFrame(Classificatioin_result)
df

,0,1,2,3
0,Invarient,2017_2019,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
1,Invarient,2014_2016,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
2,Invarient,2011_2013,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
3,Invarient,2008_2010,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
4,Invarient,2005_2007,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
5,Invarient,2002_2004,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
6,Invarient,1999_2001,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
7,Invarient,1996_1998,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
8,Invarient,1993_1995,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
9,Invarient,1990_1992,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."


In [38]:
Map = geemap.Map()
Map.addLayer(Classificatioin_result[0][2],{'min':0,'max':1}, 'Invarient 2017-2019')
Map.addLayer(Classificatioin_result[10][2],{'min':0,'max':1},'Each_year 2017-2019')
Map.addLayer(Classificatioin_result[15][2],{'min':0,'max':1},'Zone_invarient 2017-2019')
Map

Map(center=[40, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…